## Function below extracts results from csv file and let's you choose if you want the actual walking distances from 2016 or if you want the optimized walking distances from the model. Note: This code is a big slop show so tread carefully!

In [20]:
import pandas as pd
import numpy as np

def get_walking_distances(file_path = "Optimized_Gate_Assignments_Sample_Day.csv", actual_or_optimized = "optimized", tiered = False):
    if actual_or_optimized == "actual":
        DepGate = "DepGate"
        ArrGate = "ArrGate"
    else:
        DepGate = "OptDepGate"
        ArrGate = "OptArrGate"

    print("\nMetrics based on optimized gate assignments:\n")

    df = pd.read_csv(file_path)
    num_flights = df.shape[0]

    # Load the walking distances for arriving and departing flights
    walking_distances = pd.read_csv("Data/Walking Distances Arriving and Departing Pax.csv")

    # Initialize accumulators for departing and arriving walking distances and passenger counts
    total_departing_wd = 0.0
    total_arriving_wd = 0.0
    total_departing_passengers = 0
    total_arriving_passengers = 0

    # Process each flight row for departing/arriving
    for _, row in df.iterrows():
        if row["IsDeparting"] == "Y":
            # Use DepGate for departing flights
            gate = row[DepGate]
            if pd.isna(gate):
                # Gate assignment is missing; skip or warn as needed
                print(f"WARNING: Departing gate assignment is missing for flight row:\n{row}")
                continue

            gate_match = walking_distances[walking_distances["Gate_Name"] == gate]
            if gate_match.empty:
                print(f"WARNING: Departing gate '{gate}' not found in walking_distances.")
                continue

            distance = gate_match.iloc[0]["TSA_to_Gate"]
            total_departing_wd += row["PassengersDept"] * distance
            total_departing_passengers += row["PassengersDept"]

        else:
            # Use ArrGate for arriving flights
            gate = row[ArrGate]
            if pd.isna(gate):
                # Gate assignment is missing; skip or warn as needed
                print(f"WARNING: Arriving gate assignment is missing for flight row:\n{row}")
                continue

            gate_match = walking_distances[walking_distances["Gate_Name"] == gate]
            if gate_match.empty:
                print(f"WARNING: Arriving gate '{gate}' not found in walking_distances.")
                continue

            distance = gate_match.iloc[0]["Gate_to_Bag"]
            total_arriving_wd += row["PassengersArr"] * distance
            total_arriving_passengers += row["PassengersArr"]

    # Compute averages
    avg_departing_wd = (
        total_departing_wd / total_departing_passengers
        if total_departing_passengers > 0
        else 0
    )
    avg_arriving_wd = (
        total_arriving_wd / total_arriving_passengers
        if total_arriving_passengers > 0
        else 0
    )

    # ----- CONNECTIONS -----
    # Load the connections matrix (adjusting indices).
    conn_mat = pd.read_csv("Data/connections_matrix.csv", header=None)
    conn_tier_mat = pd.read_csv("Data/connections_tier_matrix.csv", header=None)
    # Slice out the relevant portion for your flights
    conn_mat = conn_mat.iloc[1:num_flights+1, 1:num_flights+1].to_numpy()
    conn_tier_mat = conn_tier_mat.iloc[1:num_flights+1, 1:num_flights+1].to_numpy()
    
    # Load walking distances between gates.
    # This CSV is expected to have a "Gate_Name" column and other columns with gate names.
    walking_distances_gate_to_gate = pd.read_csv("Data/Walking Distances Gate-to-Gate.csv")

    # For referencing gate "integers", we use the same walking_distances DataFrame,
    # which has columns ["Gate_Name", "Gate_Int", "TSA_to_Gate", "Gate_to_Bag"]
    # We'll build a dictionary mapping gate name -> integer for faster lookups:
    gate_to_int = dict()
    for _ , row_w in walking_distances.iterrows():
        gate_name = row_w["Gate_Name"]
        gate_int  = int(row_w["Gate_Int"])
        gate_to_int[gate_name] = gate_int

    if not tiered:
        total_connection_wd = 0.0
        total_connection_passengers = 0

        # Loop over flight pairs in the connection matrix
        num_rows, num_cols = conn_mat.shape

        for i in range(num_rows):
            for j in range(num_cols):
                num_connect = conn_mat[i, j]
                if num_connect > 0:
                    flight_i = df.iloc[i]
                    flight_j = df.iloc[j]

                    # flight_i is the inbound flight => use ArrGate
                    gate_i_name = flight_i[ArrGate]
                    # flight_j is the outbound flight => use DepGate
                    gate_j_name = flight_j[DepGate]

                    # If either is missing, skip
                    if pd.isna(gate_i_name) or pd.isna(gate_j_name):
                        continue

                    # Check if the name is in the gate_to_int dict
                    if gate_i_name not in gate_to_int:
                        print(f"WARNING: Arriving gate '{gate_i_name}' not found in gate_to_int dictionary.")
                        continue
                    if gate_j_name not in gate_to_int:
                        print(f"WARNING: Departing gate '{gate_j_name}' not found in gate_to_int dictionary.")
                        continue

                    gate_i_int = gate_to_int[gate_i_name]
                    gate_j_int = gate_to_int[gate_j_name]

                    gate_to_gate_wd = walking_distances_gate_to_gate.iloc[gate_i_int-1, gate_j_int-1]

                    # Add to total
                    total_connection_wd += num_connect * gate_to_gate_wd
                    total_connection_passengers += num_connect

        # Compute average connection distance
        avg_connection_wd = (
            total_connection_wd / total_connection_passengers
            if total_connection_passengers > 0
            else 0
        )

        print("Average Departing Passenger Walking Distance:", round(avg_departing_wd, 2))
        print("Average Arriving  Passenger Walking Distance:", round(avg_arriving_wd, 2))
        print("Average Connecting Passenger Walking Distance:", round(avg_connection_wd, 2))
        print("Total flights:", num_flights)
        print("Total Departing Passengers:", total_departing_passengers)
        print("Total Arriving Passengers:", total_arriving_passengers)
        print("Total Connecting Passengers:", total_connection_passengers)
    else:
        total_connection_wd_tier1 = 0.0
        total_connection_passengers_tier1 = 0
        total_connection_wd_tier2 = 0.0
        total_connection_passengers_tier2 = 0
        total_connection_wd_tier3 = 0.0
        total_connection_passengers_tier3 = 0

        # Loop over flight pairs
        num_rows, num_cols = conn_mat.shape
        for i in range(0, num_rows):
            for j in range(0, num_cols):
                num_connect = conn_mat[i, j]
                if num_connect > 0:
                    # Get tier of these connecting passengers
                    tier = conn_tier_mat[i, j]

                    flight_i = df.iloc[i]
                    flight_j = df.iloc[j]
                    
                    # Determine gate assignment based on flight type
                    gate_i_name = flight_i[ArrGate] # Flight i is always inbound
                    gate_j_name = flight_j[DepGate] # Flight j is always outbound

                    # If either is missing, skip
                    if pd.isna(gate_i_name) or pd.isna(gate_j_name):
                        continue

                    # Check if the name is in the gate_to_int dict
                    if gate_i_name not in gate_to_int:
                        print(f"WARNING: Arriving gate '{gate_i_name}' not found in gate_to_int dictionary.")
                        continue
                    if gate_j_name not in gate_to_int:
                        print(f"WARNING: Departing gate '{gate_j_name}' not found in gate_to_int dictionary.")
                        continue

                    gate_i = gate_to_int[gate_i_name] # Get the int from the gate name
                    gate_j = gate_to_int[gate_j_name] # Get the int from the gate name
                    
                    gate_to_gate_wd = walking_distances_gate_to_gate.iloc[gate_i-1, gate_j-1]

                    if tier == 1:
                        total_connection_wd_tier1 += num_connect * gate_to_gate_wd
                        total_connection_passengers_tier1 += num_connect
                    
                    if tier == 2:
                        total_connection_wd_tier2 += num_connect * gate_to_gate_wd
                        total_connection_passengers_tier2 += num_connect

                    if tier == 3:
                        total_connection_wd_tier3 += num_connect * gate_to_gate_wd
                        total_connection_passengers_tier3 += num_connect

        avg_connection_wd_tier1 = total_connection_wd_tier1 / total_connection_passengers_tier1
        avg_connection_wd_tier2 = total_connection_wd_tier2 / total_connection_passengers_tier2
        avg_connection_wd_tier3 = total_connection_wd_tier3 / total_connection_passengers_tier3

        print("Average Departing Passenger Walking Distance:", round(avg_departing_wd, 2))
        print("Average Arriving Passenger Walking Distance:", round(avg_arriving_wd, 2))
        print("Average Tier 1 Connecting Passenger Walking Distance:", round(avg_connection_wd_tier1, 2))
        print("Average Tier 2 Connecting Passenger Walking Distance:", round(avg_connection_wd_tier2, 2))
        print("Average Tier 3 Connecting Passenger Walking Distance:", round(avg_connection_wd_tier3, 2))
        print("Total flights:", num_flights)
        print("Total Departing Passengers:", total_departing_passengers)
        print("Total Arriving Passengers:", total_arriving_passengers)
        print("Total Tier 1 Connecting Passengers:", total_connection_passengers_tier1)
        print("Total Tier 2 Connecting Passengers:", total_connection_passengers_tier2)
        print("Total Tier 3 Connecting Passengers:", total_connection_passengers_tier3)

In [22]:
get_walking_distances("Optimized_Gate_Assignments_Sample_Day.csv", "actual", tiered = False)


Metrics based on optimized gate assignments:



/var/folders/6f/0t_pqxjx06q6_18yc4lp8j3w0000gn/T/ipykernel_7636/1968791349.py:76: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  conn_mat = pd.read_csv("Data/connections_matrix.csv", header=None)
/var/folders/6f/0t_pqxjx06q6_18yc4lp8j3w0000gn/T/ipykernel_7636/1968791349.py:77: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  conn_tier_mat = pd.read_csv("Data/connections_tier_matrix.csv", header=None)


Average Departing Passenger Walking Distance: 109.36
Average Arriving  Passenger Walking Distance: 160.35
Average Connecting Passenger Walking Distance: 764.38
Total flights: 1200
Total Departing Passengers: 45924
Total Arriving Passengers: 42377
Total Connecting Passengers: 17633
